# Домашнее задание по обработке текстов

Входные данные:
* Новости о компании "Газпром" с 2010 года

In [7]:
with open('gazprom.txt') as f:
    raw = f.read()
    
news = raw.split('--------------------------------------------------------------------------')

len(news)

2393

In [8]:
news[-1]

'\n"Газпрому" не придется доплачивать за транзит газа через Украину\nАвторы: Vedomosti\nОпубликовано 04 Января 2010 в 23:51:00\n\nДоплата, которую "Газпром" должен сделать в связи с ростом тарифа на транзит, будет зачтена при оплате за приобретенный НАК "Нафтогаз Украины" газ, сообщил заместитель главы НАК Вадим Чупрун.'

In [9]:
import re

def month_to_num(word):
    months = ['января','февраля','марта','апреля','мая','июня','июля','августа','сентября','октября','ноября','декабря']
    res = months.index(word)+1
    if res < 10:
        res = '0'+str(res)
    return(str(res))

def extract_date(news):
    regexp_date = re.compile('Опубликовано\s[0-9]{1,2}\s[А-Яа-я]+\s201\d') 
    date_raw = regexp_date.findall(news)[0].lower().split()[1:]
    date = str(date_raw[0])+'.'+month_to_num(date_raw[1])+'.'+str(date_raw[2])
    return(date)

extract_date(news[0])

'09.11.2017'

In [10]:
def extract_text(news):
    parts = news.split('\n')
    return (parts[0]+' '+parts[-2])

print(extract_text(news[0]))

Компания рассчитывает на решение по газовому спору с Украиной до конца декабря «Газпром» ожидает решения Стокгольмского арбитража по транзитному спору с украинским «Нафтогазом» не позднее февраля 2018 года, хотя раньше компания прогнозировала, что это произойдет до 30 ноября. Решение по взаимным искам об условиях поставок газа на Украину, по мнению монополии, суд может принять на месяц раньше — до декабря 2017 года.


In [11]:
events_by_date = {}

for n in news:
    date = extract_date(n)
    if date in events_by_date.keys():
        events_by_date[date] += ('\n'+extract_text(n))
    else:
        events_by_date[date] = extract_text(n)
    

In [50]:
import pandas as pd

df = pd.DataFrame.from_dict(events_by_date, orient='index')
df = df.reset_index()
df.columns = ['date', 'text']
df.to_csv('texts.csv', index=False)

In [53]:
df1 = pd.read_csv('texts.csv')

In [54]:
df1.head()

,date,text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [13]:
df = df.iloc[::-1]
df.head(10)

,0
04.01.2010,
11.01.2010,Спорные вопросы по оплате за оказанные в пери...
19.01.2010,"""Газпром"" готов забирать весь объем азербайдж..."
28.01.2010,"Консорциум во главе с российским ОАО ""Газпром..."
01.02.2010,"""Газпром"" не исключает в 2010 г. выпуска обли..."
03.02.2010,"Переговоры ОАО ""Газпром"" с оператором проекта..."
04.02.2010,"Российский газовый монополист – компания ""Газ..."
15.02.2010,"Российский ""Газпром"" сообщил о планах увеличе..."
10.03.2010,"Подконтрольный ""Газпрому"" нефтедобытчик ""Газп..."
22.03.2010,"""Газпром"" назвал участников конкурса по отбор..."


# Quotations

In [264]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all = pr_all.iloc[::-1]
pr_all.head(10)

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
1987,11.01.2010,"192,50000","194,50000","196,90000","191,00000",59815603
1986,12.01.2010,"194,00000","191,80000","195,80000","190,55000",43482579
1985,13.01.2010,"189,72000","189,30000","191,86000","188,30000",52416087
1984,14.01.2010,"191,20000","190,83000","191,46000","188,88000",41207109
1983,15.01.2010,"190,00000","187,86000","192,10000","187,25000",55571965
1982,18.01.2010,"186,97000","190,40000","190,50000","186,60000",36525951
1981,19.01.2010,"190,10000","189,76000","190,47000","187,36000",48626727
1980,20.01.2010,"189,09000","185,98000","189,90000","185,82000",46894531
1979,21.01.2010,"187,17000","182,76000","187,19000","182,30000",62823179
1978,22.01.2010,"181,50000","181,50000","183,00000","178,51000",62923848


In [267]:
pr = pr_all[pr_all.Date.isin(df.index)]

Для некоторых дат нет цен, удалим тексты за эти даты за датасета с новостями.

In [268]:
df = df[df.index.isin(pr.Date)]

In [269]:
pr.head(10)

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
1987,11.01.2010,"192,50000","194,50000","196,90000","191,00000",59815603
1981,19.01.2010,"190,10000","189,76000","190,47000","187,36000",48626727
1974,28.01.2010,"183,75000","182,30000","185,98000","181,67000",62561241
1972,01.02.2010,"184,74000","189,85000","190,40000","183,50000",76298175
1970,03.02.2010,"194,00000","192,70000","195,27000","192,30000",74730185
1969,04.02.2010,"192,60000","186,59000","193,39000","185,78000",72663606
1962,15.02.2010,"163,52000","164,45000","165,16000","162,50000",40268788
1947,10.03.2010,"174,80000","173,77000","177,71000","172,65000",64149524
1939,22.03.2010,"166,34000","169,29000","169,64000","164,89000",68318812
1935,26.03.2010,"161,97000","161,10000","162,74000","159,49000",79370939


In [275]:
import numpy as np

target = np.array(pr['ClosingPrice'])
target[:10]

array(['194,50000', '189,76000', '182,30000', '189,85000', '192,70000',
       '186,59000', '164,45000', '173,77000', '169,29000', '161,10000'], dtype=object)

In [276]:
len(target)

1159

In [277]:
df.shape

(1159, 1)

In [278]:
target

array(['194,50000', '189,76000', '182,30000', ..., '126,50000',
       '132,30000', '131,50000'], dtype=object)

In [279]:
def price(s):
    s1 = str(s)
    s = s1.replace(',','.')
    return(float(s))
    
   
for i in range(1,len(target)):
    if price(target[i-1]) < price(target[i]):
        target[i-1] = 0
    else:
        target[i-1] = 1

      
target[:10]

array([1, 1, 0, 0, 1, 1, 0, 1, 1, 0], dtype=object)

In [280]:
target[-1] = 1
target
y = target.astype('int')

In [281]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
df1 = vectorizer.fit_transform(df[0])

In [282]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df1, y, test_size=0.33, random_state=42)

In [283]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LogReg = LogisticRegression()
preds = LogReg.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, preds))

             precision    recall  f1-score   support

          0       0.46      0.48      0.47       169
          1       0.58      0.56      0.57       214

avg / total       0.53      0.52      0.53       383

